In [2]:
import tensorflow as tf
import torch as to
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class Membrane_PINN(nn.Module):
    def __init__(self, ics, radius=1, c2 = 1,
                 layers = [3, 64, 64, 32, 32, 16, 16, 8, 8, 2], 
                 activation=nn.ReLU()
                 ):
        """
        Parameters
        -------------
        layers: Set to default [3, 64, 64, 32, 32, 16, 16, 8, 8, 2] 
                where the 0th value is input and the -1th value is output
        radius: the radius of the membrane.
        ics: the initial condition of the membrane, typically a function of r, theta 
        c2: wave speed squared/
        activation: The activation function you wish to use in the network
        """
        super().__init__()
        network = []
        for i in range(len(layers)-1):
            network.append(nn.Linear(layers[i],layers[i+1]))
            network.append(activation)
        self.output = nn.Sequential(*network) # our way of making a dynamic network of any size :)
        self.ics = ics
        self.bcs = 0
        self.depth = len(layers) - 1
        self.activation = activation
        self.radius = radius
        self.c2 = c2

    
    def forward(self, x):
        return self.output(x)
    

    # def grad(outputs, inputs):
    #     """Computes the partial derivative of 
    #     an output with respect to an input.
    #     Args:
    #         outputs: (N, 1) tensor
    #         inputs: (N, D) tensor
    #     """
    #     return torch.autograd.grad(
    #         outputs, inputs, grad_outputs=torch.ones_like(outputs), create_graph=True
    # )


    def get_pde_residual(self, xi, r, theta, t):
        xi_r = tf.gradients(xi, r)
        rxi_r = r * xi_r
        rxi_r_r = tf.gradients(rxi_r, r)
        rxi_r_r_over_r = rxi_r_r / r

        xi_theta = tf.gradients(xi, theta)
        xi_ttheta = tf.gradients(xi_theta, theta)
        xi_ttheta_over_rr = xi_ttheta / r**2

        xi_t = tf.gradients(xi, t)
        xi_tt = tf.gradients(xi_t, t)
        residual = xi_tt - self.c2 * (xi_ttheta_over_rr + rxi_r_r_over_r)
        
        return residual


    def backward(self):
        pass


    def train(self, X, Y):
        pass


    def predict(self):
        pass



def main():
    pass


if __name__ == "__main__":
    main()
